## Desafio 2


Dado uma imagem de captcha (no repositório, “imagens/dots.png”) obtenha o contorno dos caracteres e o exibia na imagem original.
Realize a seguinte ordem: suavização, limiarização, dilatação/erosão, detecção de contornos.
Bônus: use a função de enlaçamento de retângulo para extrair as letras.

![Captcha](imagens/captcha-2.jpg)

In [189]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [190]:
image = cv2.imread("imagens/captcha-2.jpg", cv2.IMREAD_GRAYSCALE)
cv2.imshow("Captcha", image)
cv2.waitKey()
cv2.destroyAllWindows()

Aplicando suavização para posterior limiarização

In [191]:
image_blur = cv2.blur(image, (5, 5))
cv2.imshow("Old Person Blur", image_blur)
cv2.waitKey()
cv2.destroyAllWindows()

Aplicando limiarização para extração de ruídos

In [192]:
ret, t_image = cv2.threshold(image_blur, 140, 255, cv2.THRESH_BINARY)

cv2.imshow("Captcha T", t_image)
cv2.waitKey()
cv2.destroyAllWindows()

Erodindo para enaltecer as letras. Lembrando que preto é ausência de informação logo, erodir significa abrir e "engrossa" as letras

In [193]:
kernel = np.ones((5,5), np.uint8)
image_eroded = cv2.erode(t_image, kernel, iterations=1)

cv2.imshow("Captcha Td", image_eroded)
cv2.waitKey()
cv2.destroyAllWindows()

Aplicando filtro Canny para detecção de bordas

In [194]:
image_edged = cv2.Canny(image_eroded, 30, 180)
cv2.imshow("Formas 2 Edged", image_edged)
cv2.waitKey()
cv2.destroyAllWindows()

Dilatando as bordas para melhorar a detecção de contornos

In [195]:
kernel_d = np.ones((1,1), np.uint8)
image_dilated = cv2.dilate(image_edged, kernel_d, iterations=1)
cv2.imshow("Captcha D", image_dilated)
cv2.waitKey()
cv2.destroyAllWindows()

In [196]:
_, contours, hierarchy = cv2.findContours(image_dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print("Contornos encontrados "+str(len(contours)))


Contornos encontrados 4


Se fossem encontrados mais do que as 4 letras poderíamos aplicar algum filtro de limite de tamnaho de área

In [197]:
contours_filtered = []
for idx, contour in enumerate(contours):
    area = cv2.contourArea(contour)
    if area > 100:
        contours_filtered.append(contour)
        (x, y, w, h) = cv2.boundingRect(contour)
        print(str(area))
        print((x, y, w, h))
len(contours_filtered)

702.5
(273, 42, 28, 31)
951.0
(204, 42, 34, 39)
718.5
(239, 41, 30, 32)
442.0
(308, 38, 25, 35)


4

Desenhando os contornos na imagem original

In [198]:
image_original = cv2.imread("imagens/captcha-2.jpg")
cv2.drawContours(image_original, contours_filtered, -1, (0,255,0), 2)
cv2.imshow("Formas 2 Contours", image_original)
cv2.waitKey()
cv2.destroyAllWindows()

Extraindo letras para eventual treinamento em alguma algoritmo de rede neural, por exemplo

In [199]:
for idx, contour in enumerate(contours_filtered):
    (x, y, w, h) = cv2.boundingRect(contour)
    letter_image = image_edged[y - 10:y + h + 30, x - 1:x + w + 1]
    cv2.imwrite(str(idx) + ".png", letter_image)
    cv2.rectangle(image_original, (x,y), (x+w,y+h), (0, 255, 0), 2)
    print(x, y, w, h)
    
    cv2.imshow("Formas 2 Contours", letter_image)
    cv2.waitKey()
    cv2.destroyAllWindows()

273 42 28 31
204 42 34 39
239 41 30 32
308 38 25 35
